# Experiment with CIFAR10
## Federated learning with random selection 3

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['AUTHGRAPH_VERBOSITY'] = '0'
import pickle
import time
import datetime
import random
import operator
import copy

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.random.set_seed(6292)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import tensorflow_hub as hub
import numpy as np
import pandas as pd